# ПАРСЕР ДЛЯ САЙТА ПОСТУПИ.ОНЛАЙН:
Скачиваем данные по таким категориям, как 'Название программы', 'Бюджетные', 'Платные', 'Стоимость обучения', 'Направление подготовки'

In [51]:
import requests
from bs4 import BeautifulSoup

# URL of the page to parse
url = "https://msk.postupi.online/vuz/ranhigs/programmy-obucheniya/magistratura/?utm_module=vuz&utm_vuz=8"

# Initialize an empty list to store the parsed data
parsed_data = []

# Iterate over each page
for page_num in range(0, 8):
    # Send a GET request to the URL with the current page number
    response = requests.get(f"{url}&page_num={page_num}&utm_module=vuz&utm_vuz=8")

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all list items with the class "list"
    list_items = soup.find_all('li', class_='list')

    # Iterate over each list item
    for item in list_items:
        # Extract the program name
        program_name = item.find('h2', class_='list__h').text.strip()

        info = item.find_all('p', class_='list__score')
        inn1 = info[0].text
        inn2 = info[1].text

        # Extract the number of budgetary and paid places
        budget_places = inn1.strip().split()[3]
        paid_places = inn2.strip().split()[3]


        # Extract the cost of education
        cost_element = item.find('span', class_='list__price')
        if cost_element:
            cost = cost_element.text.strip().replace('от', '').replace('<b>', '').replace('</b>', '').replace(' руб.', '').replace(' ', '').replace('Информация о стоимости обучения предоставлена за', '').replace('год', '').replace(' ', '')
            cost = cost[:6]
        else:
            cost = ''

        # Extract the form of preparation
        form = item.find('p', class_='list__pre').text.strip().split()[0]

        # Create a dictionary to store the parsed data
        parsed_item = {
            'Название программы': program_name,
            'Бюджетные': budget_places,
            'Платные': paid_places,
            'Стоимость обучения': cost,
            'Направление подготовки': form
        }

        # Add the parsed item to the list
        parsed_data.append(parsed_item)

# Save the parsed data to a CSV file
with open('parsed_data00.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Название программы', 'Бюджетные', 'Платные', 'Стоимость обучения', 'Направление подготовки']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for item in parsed_data:
        writer.writerow(item)

print("Data saved to parsed_data00.csv")

Data saved to parsed_data00.csv
